In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [7]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import gc
import faiss

In [3]:
train = pd.read_parquet('df_vector.pq')
test = pd.read_parquet('df_vector_test.pq')

In [4]:
train.head(2)

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,key_list,name_list,text_vector
4,2700005,Разработчик Python/django,"[137, 221, 296]",,845fe6266ab1,[1],небольшой дружный команда требоваться опытный ...,[],"[Разработчик, Python/django]","[-0.37388623, 0.26167643, 0.3595109, -0.166157..."
5,2700006,Торговый представитель,"[149, 242]",работа текущий база клиент мерчендайзинг работ...,23ead4a8f164,[17],обязанность компания дистрибьютор продукт пита...,"[работа, текущий, база, клиент, мерчендайзинг,...","[Торговый, представитель]","[-0.07712369, -0.11375963, 0.07628894, -0.0080..."


In [5]:
test.head(2)

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,key_list,name_list,text_vector
0,2700001,прораб великий устюг,None,,72dc8d78bbec,None,обязанность взаимодействие субподрядчик предос...,[],"[прораб, великий, устюг]","[-0.050694242, 0.4064839, 0.55667454, -0.16124..."
1,2700002,лаборант электротехнический лаборатория,None,office пользователь деловой общение деловой пе...,ba6dcbd0bcd1,None,связь расширение компания требоваться quot лаб...,"[office, пользователь, деловой, общение, делов...","[лаборант, электротехнический, лаборатория]","[-0.19709699, -0.35366777, 0.10756227, 0.23191..."


In [40]:
train_vectors = dill.load(open('train_vectors.bin', 'rb'))
test_vectors = dill.load(open('test_vectors.bin', 'rb'))
train_specs = dill.load(open('train_spec.bin', 'rb'))

In [41]:
test_vectors = np.array(test_vectors)

In [42]:
print( train_vectors.shape, test_vectors.shape, len(train_specs))

(1456325, 400) (1456325, 400) 1456325


In [10]:
# res = faiss.StandardGpuResources()
# flat_config = faiss.GpuIndexFlatConfig()
# flat_config.device = 0
# faiss.gpu_profiler_start()
# index = faiss.GpuIndexFlatL2(res, 300, flat_config)

In [43]:
index = faiss.IndexFlatL2(400)

In [44]:
index.metric_type = faiss.METRIC_INNER_PRODUCT

In [45]:
%%time
faiss.normalize_L2(train_vectors)
faiss.normalize_L2(test_vectors)

CPU times: user 1.82 s, sys: 8.45 ms, total: 1.82 s
Wall time: 152 ms


In [46]:
index.add(train_vectors)

In [47]:
print(index.ntotal)

1456325


In [48]:
np.savez_compressed('norm_vectors', train=train_vectors, test=test_vectors)

In [51]:
%%time
D, I = index.search(test_vectors[:10000], 10)

CPU times: user 8min 57s, sys: 1min 14s, total: 10min 12s
Wall time: 1min


In [53]:
1456325/10000*60/60

145.6325

In [36]:
D[0]

NameError: name 'D' is not defined

In [ ]:
I[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([test_vectors[0]], [train_vectors[752562]])

In [26]:
test_indexes = dill.load(open('test_indexes.bin', 'rb'))
train_indexes = dill.load(open('train_indexes.bin', 'rb'))

In [27]:
test_ind = test_indexes[1]
test_ind

2700002

In [28]:
train_ind = train_indexes[752562]
train_ind

'1295984'

In [29]:
test[test.vacancy_id == 2700002].processed_text.values[0]

'связь расширение компания требоваться quot лаборант электротехнический лаборатория quot компания swg производитель поставщик светодиодный оборудование компания основывать год данный момент являться крупный оптовый поставщик современный интерьерный освещение россия требование образование низко средний полный средний специальный знание основа электротехника электроника уверенный пользователь office коммуникабельность умение общаться человек находить общий язык умение работать большой количество информация дополнительный преимущество знание английский язык опыт работа опыт монтаж светотехнический оборудование качество который должный обладать успешный кандидат заинтересованность работать электрооборудование общий светодиодный оборудование частность быстрый обучаемость ответственность внимательность стрессоустойчивость аналитический мышление самостоятельность умение работать режим многозадачность хороший коммуникативный навык обязанность диагностика неисправный электротехнический светодио

In [30]:
train[train.vacancy_id.isin([train_indexes[id] for id  in I[1]])].processed_text.values

array(['крупный энергосбытовой компания quot энергосбыт плюс quot приглашать кандидат вакансия инженер обязанность подготовка уведомление ограничение электроснабжение оперативный переговоры потребитель сетевой организация ведение отчетность работа претензия качество требование высокий образование электротехнический желательный опыт работа аналогичный должность опыт работа действующий электроустановка уверенный пользователь офисный программа word excel оutlook умение читать схема знание устройство электрический сеть желательно наличие группа допуск электробезопасность низкий уверенный пользователь условие официальный трудоустройство согласный медицинский страховка белый',
       'обязанность разработка новый модернизация существующий светодиодный светильник соответствие технический задание проработка отдельный компонент светильник механика оптика схема питание печатный плата теплоотвод соответствующий сапр составление технический документация разрабатывать продукция подбор закупка компл

In [31]:
train[train.vacancy_id.isin([train_indexes[id] for id  in I[0]])][['vacancy_id','specializations']]

,vacancy_id,specializations
250939,40368,[387]
268982,59375,[387]
390026,187120,[387]
612206,404990,"[287, 387]"
898616,690584,"[83, 189, 287, 387]"
1503738,1295984,"[83, 189, 287, 387]"
1657238,1457722,[387]
2090700,1882204,"[83, 189, 287, 387]"
2808874,2606514,"[83, 189, 287]"
2833463,2632408,"[83, 287]"


In [33]:
test_dict = {e:i for i,e in enumerate(test_indexes)}

In [35]:
def get_first_spec(vacancy_id):
    vind = test_dict[vacancy_id]
    return train_specs[int(train_indexes[I[vind][0]])]

In [36]:
def get_all_specs(vacancy_id):
    Is = I[test_dict[vacancy_id]]
    
    e = []
    l = []
    for sim in Is:
        elm = train_specs[int(train_indexes[sim])]
        l.append(len(elm))
        e += list(elm)
    c = Counter(e)
    return [k for k, _ in c.most_common(int(round(np.mean(l),0)))]

In [61]:
vacancy_id = 2700002
from collections import defaultdict
Is = I[test_dict[vacancy_id]]
Ds = D[test_dict[vacancy_id]]
e= []
l = []
for s_ind, sim in enumerate(Is):
    elm = train_specs[int(train_indexes[sim])]
    l.append(len(elm))
    e += [(eo, Ds[s_ind]) for eo in elm]
dd= defaultdict(float)
for spec, wght in e:
    dd[spec] +=  wght
#int(round(np.average(l, weights=Ds),0))
[s for s,w in sorted(dd.items(), key=lambda item: item[1], reverse = True)[:int(round(np.average(l, weights=Ds),0))]]

[396, 360, 81, 83]

In [79]:
def get_weigted_specs(vacancy_id):
    Is = I[test_dict[vacancy_id]]
    Ds = D[test_dict[vacancy_id]]
    e= []
    l = []
    for s_ind, sim in enumerate(Is):
        elm = train_specs[int(train_indexes[sim])]
        l.append(len(elm))
        e += [(eo, Ds[s_ind]) for eo in elm]
    dd= defaultdict(float)
    for spec, wght in e:
        dd[spec] +=  wght
    #int(round(np.average(l, weights=Ds),0))
    return [s for s,w in sorted(dd.items(), key=lambda item: item[1], reverse = True)[:int(round(np.average(l, weights=Ds),0))]]

In [83]:
def get_weigted_specs1(vacancy_id):
    Is = I[test_dict[vacancy_id]]
    Ds = D[test_dict[vacancy_id]]
    e= []
    l = []
    for s_ind, sim in enumerate(Is):
        elm = train_specs[int(train_indexes[sim])]
        l.append(len(elm))
        e += [(eo, Ds[s_ind]) for eo in elm]
    dd= defaultdict(float)
    for spec, wght in e:
        dd[spec] +=  wght
    #int(round(np.average(l, weights=Ds),0))
    return [s for s,w in sorted(dd.items(), key=lambda item: item[1], reverse = True)[:5]]

In [80]:
get_weigted_specs(2700002)

[396, 360, 81, 83]

In [81]:
get_weigted_specs1(2700002)

[396, 360, 81, 83, 70]

In [64]:
get_all_specs(2700002)

[396, 360, 81, 83]

In [82]:
get_first_spec(2700002)

array([ 87, 185, 340, 360, 381], dtype=int64)

In [71]:
data_path = '..\\Data\\In\\'
submissions_path =  '..\\Submissions\\'

In [84]:
sub = []
for vacancy_id in tqdm(test_indexes, total = len(test_indexes)):
    sub.append({'vacancy_id': vacancy_id , 'specializations': get_weigted_specs1(vacancy_id)})

In [88]:
sample_submission = pd.DataFrame(sub)

In [86]:
sample_submission['specializations'] = sample_submission['specializations'].apply(lambda x: x[:3])

In [77]:
# %%time
# sample_submission = pd.DataFrame([
#     (vacancy_id, get_first_spec(vacancy_id))
#     for vacancy_id in test_indexes
# ], columns=['vacancy_id', 'specializations'])
# sample_submission.to_csv(submissions_path+f'sample_submission_{"2020-05-25"}_{1}.csv.gz', index=False, compression='gzip')

Wall time: 2.98 s


In [89]:
sample_submission.head()

,vacancy_id,specializations
0,2700001,"[387, 287, 83, 189]"
1,2700002,"[396, 360, 81, 83, 70]"
2,2700003,"[396, 490, 515, 560, 581]"
3,2700004,"[221, 395, 117, 82, 272]"
4,2700009,"[514, 490, 287, 142, 290]"


In [90]:
sample_submission.loc[0].specializations

[387, 287, 83, 189]

In [91]:
sample_submission.to_csv(submissions_path+f'submission_{"2020-07-23"}_{"v5"}.csv.gz', index=False, compression='gzip')